In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [3]:
import tensorflow as tf
from malaya.train.model.bigbird import modeling, utils

In [4]:
bert_config = {
    'attention_probs_dropout_prob': 0.1,
    'hidden_act': 'gelu',
    'hidden_dropout_prob': 0.1,
    'hidden_size': 256,
    'initializer_range': 0.02,
    'intermediate_size': 1024,
    'max_position_embeddings': 2048,
    'max_encoder_length': 1024,
    'max_decoder_length': 1024,
    'num_attention_heads': 4,
    'num_hidden_layers': 2,
    'type_vocab_size': 2,
    'scope': 'bert',
    'use_bias': True,
    'rescale_embedding': False,
    'vocab_model_file': None,
    'attention_type': 'block_sparse',
    'block_size': 16,
    'num_rand_blocks': 3,
    'vocab_size': 32000,
    'couple_encoder_decoder': False,
    'beam_size': 1,
    'alpha': 0.0,
    'label_smoothing': 0.1,
    'norm_type': 'postnorm',
}


In [5]:
import sentencepiece as spm

vocab = 'sp10m.cased.translation.model'
sp = spm.SentencePieceProcessor()
sp.Load(vocab)

class Encoder:
    def __init__(self, sp):
        self.sp = sp
    
    def encode(self, s):
        return self.sp.EncodeAsIds(s) + [1]
    
    def decode(self, ids, strip_extraneous=False):
        return self.sp.DecodeIds(list(ids))
    
encoder = Encoder(sp)

In [6]:
model = modeling.TransformerModel(bert_config)

In [7]:
X = tf.placeholder(tf.int32, [None, None])

In [8]:
r = model(X, training = False)
r

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
reduction_indices is deprecated, use axis instead


Instructions for updating:
reduction_indices is deprecated, use axis instead


((<tf.Tensor 'bert/log_probs:0' shape=(?, 1024) dtype=float32>,
  <tf.Tensor 'bert/logits:0' shape=(?, 1024, 32000) dtype=float32>,
  <tf.Tensor 'bert/while/Exit_1:0' shape=(?, 1024) dtype=int32>),
 <tf.Tensor 'bert/encoder/layer_1/output/LayerNorm/batchnorm/add_1:0' shape=(?, 1024, 256) dtype=float32>)

In [9]:
logits = tf.identity(r[0][2], name = 'logits')
logits

<tf.Tensor 'logits:0' shape=(?, 1024) dtype=int32>

In [10]:
ckpt_path = tf.train.latest_checkpoint('bigbird-small-ms-en')
ckpt_path

'bigbird-small-ms-en/model.ckpt-470000'

In [11]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [12]:
saver = tf.train.Saver()
saver.restore(sess, ckpt_path)

INFO:tensorflow:Restoring parameters from bigbird-small-ms-en/model.ckpt-470000


INFO:tensorflow:Restoring parameters from bigbird-small-ms-en/model.ckpt-470000


In [13]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [14]:
import re
from unidecode import unidecode

def cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string.replace('\n', ' '))).strip()

In [15]:
string = """
KUALA LUMPUR: Perseteruan antara dua bekas Perdana Menteri, Tun Dr Mahathir Mohamad dan Datuk Seri Najib Tun Razak belum ada penghujungnya dengan masing-masing berbalas kenyataan di media sosial.
Selepas Najib menyanggah kenyataan tidak campur tangan dalam badan kehakiman negara dan mentertawakannya, Dr Mahathir membalasnya dengan meminta Ahli Parlimen Pekan itu memberi perhatian kepada kes 1Malaysia Development Berhad (1MDB).
Dr Mahathir juga secara sinis berkata, jika Najib boleh mengingati isu yang berlaku pada 1987 dan 1988 - isu pemilihan UMNO dan pengharaman parti itu, Najib juga boleh mengingati peristiwa dia ingin mandikan keris dengan darah.
"Saya rasa Najib tak payah campur tangan dengan tuduhan terhadap saya. Dia harus fokus kes curi duit rakyat berbilion-bilion dalam 1MDB.
"Dia juga perlu bagi perhatian saman Tommy Thomas yang kait dia dengan pembunuhan Altantuya. Lagipun, kalau isu yang berlaku 1987/88, Najib boleh ingat (peristiwa) yang dia 'hunus' keris," kata Dr Mahathir.
Sebelum ini, Najib menyanggah dakwaan Dr Mahathir yang mendakwa pembatalan pendaftaran UMNO pada 1998 sebagai bukti tidak campur tangan dalam badan kehakiman negara.
Menyokong hujahnya, Najib berkongsi apa yang berlaku pada tahun tersebut hingga menyebabkan UMNO diharamkan dan tertubuhnya UMNO baharu.
"""
cleaning(string)

'KUALA LUMPUR: Perseteruan antara dua bekas Perdana Menteri, Tun Dr Mahathir Mohamad dan Datuk Seri Najib Tun Razak belum ada penghujungnya dengan masing-masing berbalas kenyataan di media sosial. Selepas Najib menyanggah kenyataan tidak campur tangan dalam badan kehakiman negara dan mentertawakannya, Dr Mahathir membalasnya dengan meminta Ahli Parlimen Pekan itu memberi perhatian kepada kes 1Malaysia Development Berhad (1MDB). Dr Mahathir juga secara sinis berkata, jika Najib boleh mengingati isu yang berlaku pada 1987 dan 1988 - isu pemilihan UMNO dan pengharaman parti itu, Najib juga boleh mengingati peristiwa dia ingin mandikan keris dengan darah. "Saya rasa Najib tak payah campur tangan dengan tuduhan terhadap saya. Dia harus fokus kes curi duit rakyat berbilion-bilion dalam 1MDB. "Dia juga perlu bagi perhatian saman Tommy Thomas yang kait dia dengan pembunuhan Altantuya. Lagipun, kalau isu yang berlaku 1987/88, Najib boleh ingat (peristiwa) yang dia \'hunus\' keris," kata Dr Maha

In [16]:
string3 = """
Penubuhan universiti sukan seperti diutarakan Ketua Unit Sukan Kementerian Pengajian Tinggi, Dr Pekan Ramli dan disokong Pakar Pembangunan Sukan dan Reakreasi Luar, Universiti Pendidikan Sultan Idris (UPSI), Prof Dr Md Amin Md Taaf seperti disiarkan akhbar ini, memberikan sinar harapan kepada kewujudan institusi sedemikian.

Ia menjadi impian atlet negara untuk mengejar kejayaan dalam bidang sukan dan kecemerlangan dalam akademik untuk menjamin masa depan lebih baik apabila bersara daripada arena sukan kelak.

Pelbagai pandangan, idea, kaedah, bukti dan cadangan dilontarkan pakar berikutan pentingnya universiti sukan yang akan memberi impak besar sama ada pada peringkat kebangsaan mahupun antarabangsa.

Negara lain sudah lama meraih laba dengan kewujudan universiti sukan seperti China, Korea, Japan, Taiwan, India dan Vietnam. Mereka menghasilkan atlet universiti yang mempamerkan keputusan cemerlang pada peringkat tinggi seperti Sukan Olimpik, Kejohanan Dunia dan Sukan Asia.

Justeru, kejayaan mereka perlu dijadikan rujukan demi memajukan sukan tanah air. Jika kita merujuk pendekatan Asia, kewujudan universiti sukan penting dan memberi kesan positif dalam melonjakkan prestasi sukan lebih optimum.

Namun, jika kita melihat pendekatan Eropah, universiti sukan bukan antara organisasi atau institusi penting yang diberi perhatian dalam menyumbang kepada pemenang pingat.

Antara isu dalam universiti sukan ialah kos tinggi, lokasi, prasarana sukan, pertindihan kursus dengan universiti sedia ada dan impak terhadap dunia sukan negara hingga mengundang persoalan kewajaran dan kerelevanan penubuhannya.

Namun sebagai bekas atlet memanah negara dan Olympian (OLY) di Sukan Olimpik 2004 di Athens, Greece serta bekas pelajar Sekolah Sukan Bukit Jalil hingga berjaya dalam dunia akademik, saya mendapati terdapat beberapa faktor sering menjadi halangan dalam rutin harian mereka.

Antaranya, faktor masa yang terpaksa bergegas menghadiri kuliah selepas tamat sesi latihan yang mengambil masa 15 hingga 20 minit dengan menunggang motosikal; kereta (20-30 minit) atau pengangkutan disediakan Majlis Sukan Negara (MSN) ke Universiti Putra Malaysia (UPM).

Jika mereka menuntut di Universiti Teknologi MARA (UiTM) atau Universiti Malaya (UM), ia mungkin lebih lama.

Walaupun di universiti tersedia dengan kemudahan kolej dan kemudahan sukan, mereka memilih pulang ke MSN untuk menjalani latihan bersama pasukan dan jurulatih di padang atau gelanggang latihan rasmi.

Ini berlanjutan selagi bergelar atlet negara yang perlu memastikan prestasi sentiasa meningkat dari semasa ke semasa tanpa mengabaikan tugas sebagai pelajar.

Alangkah baiknya jika sebahagian Sekolah Sukan Bukit Jalil itu sendiri dijadikan Kolej atau Universiti Sukan Malaysia kerana lengkap dari segi kemudahan prasarana sukannya dan proses pengajaran dan pembelajaran (PdP) dalam bidang Sains Sukan, Kejurulatihan, Pendidikan Jasmani dan setaraf dengannya.

Pengambilan setiap semester pula hanya terhad kepada atlet berstatus kebangsaan dan antarabangsa sahaja supaya hasrat melahirkan lebih ramai atlet bertaraf Olimpik mudah direalisasikan.

Contohnya, bekas atlet lompat bergalah negara, Roslinda Samsu yang juga pemenang pingat perak Sukan Asia Doha 2006 dan Penerima Anugerah Khas Majlis Anugerah Sukan KPT 2012, terpaksa mengambil masa lebih kurang sembilan tahun untuk menamatkan ijazah Sarjana Muda Pendidikan Jasmani di UPM sepanjang 14 tahun terbabit dalam sukan olahraga.

Sepanjang tempoh bergelar atlet kebangsaan dan mahasiswa, beliau juga memenangi pingat Emas Sukan SEA empat siri berturut-turut pada 2005, 2007, 2009 dan 2011.

Begitu juga atlet kebangsaan seperti Leong Mun Yee (UPM); Pandalela Renong (UM); Bryan Nickson Lomas (UM); Cheng Chu Sian (UPM); Marbawi Sulaiman (UiTM) dan Norasheela Khalid (UPM).

Jika disenaraikan, mungkin lebih ramai lagi. Namun, pernah terlintas di fikiran mengapa hanya atlet dari sukan terjun yang dapat memenangi pingat di Sukan Olimpik? Bagaimana dengan atlet lain yang juga layak secara merit? Apakah kekangan atau masalah dihadapi sebagai atlet dan mahasiswa?

Adakah kewujudan universiti sukan akan memberi impak besar kepada kemajuan sukan negara? Jika dirancang dan diatur dengan cekap dan sistematik, ia perkara tidak mustahil dicapai.
"""

In [17]:
string4 = """
KUALA LUMPUR: Tindakan sesetengah pemimpin politik mendesak supaya pengisytiharan darurat dibatalkan dan mahu Parlimen bersidang dilihat sebagai strategi untuk menyingkirkan Tan Sri Muhyiddin Yassin sebagai Perdana Menteri melalui Dewan Rakyat.

Penganalisis politik dan Pakar Geostrategis, Prof Madya Dr Azmi Hassan, berkata ini bukan masanya untuk berbuat demikian, terutama ketika negara bergelut menangani pandemik COVID-19.



"""

In [21]:
encoded = encoder.encode(f'{cleaning(string3)}') + [1]
s = pad_sequences([encoded], padding='post', maxlen = 1024)

In [22]:
%%time
l = sess.run(r[0][2], feed_dict = {X: s})

CPU times: user 1min 38s, sys: 4.19 s, total: 1min 42s
Wall time: 7.57 s


In [23]:
encoder.decode([i for i in l[0].tolist() if i > 0])

"Establishment of sports universities as stated by the Head of Sports Unit of the Ministry of Higher Education, Dr Pekan Ramli and supported by Sports Development and External Recreation Specialist, Sultan Idris University of Education (UPSI), Prof Dr Md Amin Md Taaf as published by this newspaper, gives a ray of hope to the existence of such institutions. It is the dream of national athletes to pursue success in sports and excellence in academics to ensure a better future when retiring from the sports arena later. Various views, ideas, methods, evidence and suggestions were thrown by experts following the importance of sports universities that will have a big impact either at the national and international levels. Other countries have long earned profits with the existence of sports universities such as China, Korea, Japan, Taiwan, India and Vietnam. They produce university athletes who show excellent results at a high level such as the Olympics, World Championships and Asian Games. T

In [24]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/test-ms-en.tar.gz
# !tar -zxf test-ms-en.tar.gz

In [25]:
batch_size = 24

path = 'test'

with open(os.path.join(path, 'left.txt')) as fopen:
    left = fopen.read().split('\n')
    
with open(os.path.join(path, 'right.txt')) as fopen:
    right = fopen.read().split('\n')
    
len(left), len(right)

(100000, 100000)

In [26]:
encoded = encoder.encode(left[0]) + [1]
s = pad_sequences([encoded], padding='post', maxlen = 1024)

In [27]:
p = sess.run(logits, feed_dict = {X: s}).tolist()
results = []
for row in p:
    results.append([i for i in row if i not in [0, 1]])
results

[[10730,
  1142,
  14729,
  221,
  13,
  501,
  13,
  960,
  4782,
  2005,
  10730,
  1142,
  14729,
  221,
  13,
  501,
  13,
  960,
  4782,
  2005,
  30,
  29,
  1593,
  25,
  21,
  20974,
  1724,
  22,
  21,
  20209,
  13,
  9752,
  1130,
  22,
  12816,
  156,
  1432,
  9]]

In [28]:
from tensor2tensor.utils import bleu_hook
bleu_hook.compute_bleu(reference_corpus = [encoder.encode(right[0])], 
                       translation_corpus = results)

0.97402745

In [29]:
from tqdm import tqdm

results = []
for i in tqdm(range(0, len(left), batch_size)):
    index = min(i + batch_size, len(left))
    x = left[i: index]
    encoded = [encoder.encode(l) + [1] for l in x]
    batch_x = pad_sequences(encoded, padding='post', maxlen = 1024)
    
    p = sess.run(logits, feed_dict = {X: batch_x}).tolist()
    result = []
    for row in p:
        result.append([i for i in row if i not in [0, 1]])
    results.extend(result)

100%|██████████| 4167/4167 [3:54:49<00:00,  3.38s/it]  


In [30]:
rights = [encoder.encode(r) for r in right[:len(results)]]
bleu_hook.compute_bleu(reference_corpus = rights,
                       translation_corpus = results)

0.58603466

In [31]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'output/model.ckpt')

'output/model.ckpt'

In [32]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'logits' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
        and 'gradients' not in n.name
    ]
)
strings.split(',')

['bert/embeddings/word_embeddings',
 'bert/embeddings/position_embeddings',
 'Placeholder',
 'bert/encoder/LayerNorm/gamma',
 'bert/encoder/layer_0/attention/self/query/kernel',
 'bert/encoder/layer_0/attention/self/query/bias',
 'bert/encoder/layer_0/attention/self/key/kernel',
 'bert/encoder/layer_0/attention/self/key/bias',
 'bert/encoder/layer_0/attention/self/value/kernel',
 'bert/encoder/layer_0/attention/self/value/bias',
 'bert/encoder/layer_0/attention/self/Softmax',
 'bert/encoder/layer_0/attention/self/Softmax_1',
 'bert/encoder/layer_0/attention/self/Softmax_2',
 'bert/encoder/layer_0/attention/self/Softmax_3',
 'bert/encoder/layer_0/attention/self/Softmax_4',
 'bert/encoder/layer_0/attention/output/dense/kernel',
 'bert/encoder/layer_0/attention/output/dense/bias',
 'bert/encoder/layer_0/attention/output/LayerNorm/gamma',
 'bert/encoder/layer_0/intermediate/dense/kernel',
 'bert/encoder/layer_0/intermediate/dense/bias',
 'bert/encoder/layer_0/output/dense/kernel',
 'bert/e

In [33]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [34]:
freeze_graph('output', strings)

INFO:tensorflow:Restoring parameters from output/model.ckpt


INFO:tensorflow:Restoring parameters from output/model.ckpt


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


...

INFO:tensorflow:Froze 90 variables.


INFO:tensorflow:Froze 90 variables.


INFO:tensorflow:Converted 90 variables to const ops.


INFO:tensorflow:Converted 90 variables to const ops.


4602 ops in the final graph.


In [35]:
from tensorflow.tools.graph_transforms import TransformGraph

In [36]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

In [37]:
pb = 'output/frozen_model.pb'

input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())
        
inputs = ['Placeholder']
transformed_graph_def = TransformGraph(input_graph_def, 
                                       inputs,
                                       ['logits'], transforms)

with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

Instructions for updating:
Use tf.gfile.GFile.


Instructions for updating:
Use tf.gfile.GFile.


In [38]:
def load_graph(frozen_graph_filename, **kwargs):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # https://github.com/onnx/tensorflow-onnx/issues/77#issuecomment-445066091
    # to fix import T5
    for node in graph_def.node:
        if node.op == 'RefSwitch':
            node.op = 'Switch'
            for index in xrange(len(node.input)):
                if 'moving_' in node.input[index]:
                    node.input[index] = node.input[index] + '/read'
        elif node.op == 'AssignSub':
            node.op = 'Sub'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'AssignAdd':
            node.op = 'Add'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'Assign':
            node.op = 'Identity'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
            if 'validate_shape' in node.attr:
                del node.attr['validate_shape']
            if len(node.input) == 2:
                node.input[0] = node.input[1]
                del node.input[1]

    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph


In [39]:
g = load_graph('output/frozen_model.pb')
x = g.get_tensor_by_name('import/Placeholder:0')
logits = g.get_tensor_by_name('import/logits:0')
test_sess = tf.InteractiveSession(graph = g)

In [40]:
%%time
l = test_sess.run(logits, feed_dict = {x: s})
encoder.decode([i for i in l[0].tolist() if i > 0])

CPU times: user 5.48 s, sys: 244 ms, total: 5.73 s
Wall time: 856 ms


'Maisoncelles-la-Jourdan Maisoncelles-la-Jourdan is a commune in the Calvados department of the Basse-Normandie region of northwestern France.'

In [41]:
g = load_graph('output/frozen_model.pb.quantized')
x = g.get_tensor_by_name('import/Placeholder:0')
logits = g.get_tensor_by_name('import/logits:0')
test_sess = tf.InteractiveSession(graph = g)

In [42]:
%%time
l = test_sess.run(logits, feed_dict = {x: s})
encoder.decode([i for i in l[0].tolist() if i > 0])

CPU times: user 5.94 s, sys: 325 ms, total: 6.27 s
Wall time: 1.2 s


'Maisoncelles-la-Jourdan Maisoncelles-la-Jourdan is a commune in the Calvados department of the Basse-Normandie region of northwestern France.'

In [1]:
!rm -rf bigbird-small-ms-en